In [153]:
import pandas as pd
import sqlite3
import os

os.makedirs('../exports', exist_ok=True)

conn = sqlite3.connect('../data/db/spotify.db')
print("Connected")

Connected


In [156]:
# loading datasets
spotify_clean = pd.read_csv('../data/cleaned/spotify_cleaned.csv', low_memory=False)
spotify_genres = pd.read_csv('../data/cleaned/spotify_genres.csv', low_memory=False)
spotify_artists = pd.read_csv('../data/raw/spotify_artist_data_2023.csv', low_memory=False)

spotify_clean.to_sql('spotify_clean', conn, if_exists='replace', index=False)
spotify_genres.to_sql('spotify_genres', conn, if_exists='replace', index=False)
spotify_artists.to_sql('spotify_artists', conn, if_exists='replace', index=False)

print("Datasets loaded")
print("spotify_clean rows:", pd.read_sql("SELECT COUNT(*) as count FROM spotify_clean", conn).iloc[0,0])
print("spotify_genres rows:", pd.read_sql("SELECT COUNT(*) as count FROM spotify_genres", conn).iloc[0,0])
print("spotify_artists rows:", pd.read_sql("SELECT COUNT(*) as count FROM spotify_artists", conn).iloc[0,0])

Datasets loaded
spotify_clean rows: 374364
spotify_genres rows: 683378
spotify_artists rows: 37012


In [5]:
# what year is the oldest song in the dataset from, and what year is the newest song in the dataset from?

In [88]:
query = pd.read_sql("""
    SELECT MIN(release_year) AS earliest, MAX(release_year) AS latest
    FROM spotify_clean
""", conn)
print(query)

   earliest  latest
0    1886.0  2023.0


In [89]:
# which genres in 2023 produced the most tracks?

In [90]:
q1 = pd.read_sql("""
    SELECT artist_genres as Genre,
           COUNT(*) as Track_Count,
           ROUND(AVG(track_popularity), 2) as AVG_Popularity
    FROM spotify_genres
    WHERE artist_genres != 'Unknown'
        AND release_year = 2023
    GROUP BY artist_genres
    ORDER BY track_count DESC
    LIMIT 15
""", conn)

print("Q1: Top 15 genres by track and their popularity count as of 2023")
print(q1)

Q1: Top 15 genres by track and their popularity count as of 2023
                Genre  Track_Count  AVG_Popularity
0                 pop         1170           46.90
1                 rap         1139           46.85
2             hip hop          593           49.70
3               sleep          577            0.57
4                rock          566           35.97
5           classical          547            6.50
6              trance          486           26.86
7       lo-fi product          481            0.30
8             pop rap          474           46.16
9                trap          452           41.93
10                edm          445           35.50
11          pop dance          393           33.42
12      environmental          392            0.00
13          dance pop          386           33.38
14  progressive house          384           20.59


In [73]:
# which artists were the most popular in 2023?

In [7]:
q2 = pd.read_sql("""
    SELECT name as Name,
           followers as Followers,
           artist_popularity AS Popularity_Score
    FROM spotify_artists
    GROUP BY followers
    ORDER BY popularity_score DESC
    LIMIT 20
""", conn)

print("Q2: Top 20 artists by popularity score as of 2023")
print(q2)

Q2: Top 20 artists by popularity score as of 2023
             Name  Followers  Popularity_Score
0    Taylor Swift   95859165               100
1           Drake   83298497                95
2       Bad Bunny   77931484                95
3      The Weeknd   75945958                93
4    Travis Scott   25672786                90
5      Kanye West   22582692                90
6      Peso Pluma    9775920                90
7          Eminem   79891173                89
8    Lana Del Rey   30603103                89
9       Jung Kook   11781452                89
10  Ariana Grande   95710972                88
11  Justin Bieber   75112165                88
12        Rihanna   59152035                88
13        KAROL G   42614177                88
14            SZA   16347096                88
15      21 Savage   15923488                88
16         Future   15561872                88
17       Junior H   12321049                88
18     Ed Sheeran  115998928                87
19  Billie

In [ ]:
# what were the top 20 popular tracks of 2023?

In [26]:
q3 = pd.read_sql("""
    SELECT track_name AS Track_Name,
           name AS Name
    FROM spotify_clean
    ORDER BY track_popularity DESC
    LIMIT 20
""", conn)

print('\n',"Q3: Top 20 songs by popularity score as of 2023")
print('\n',q3)


 Q3: Top 20 songs by popularity score as of 2023

                                            Track_Name               Name
0                     All I Want For Christmas Is You       Mariah Carey
1                                        Cruel Summer       Taylor Swift
2                               My Love Mine All Mine             Mitski
3                                         Perro Negro          Bad Bunny
4                                              Monaco          Bad Bunny
5                                            Que Onda           Calle 24
6                                           As It Was       Harry Styles
7   What Was I Made For? [From The Motion Picture ...      Billie Eilish
8                                             Starboy         The Weeknd
9      One Of The Girls (With Jennie, Lily Rose Depp)         The Weeknd
10  Is It Over Now? (Taylor'S Version) (From The V...       Taylor Swift
11                                   Jingle Bell Rock        Bobby Helms

In [ ]:
# what were the top 20 tracks released in 2023?

In [91]:
q4 = pd.read_sql("""
    SELECT track_name AS Track_Name,
           name AS Name
    FROM spotify_clean
    WHERE release_year = 2023
    ORDER BY track_popularity DESC
    LIMIT 20
""", conn)

print('\n',"Q4: Top 20 songs by popularity score as of 2023")
print('\n',q4)


 Q4: Top 20 songs by popularity score as of 2023

                                            Track_Name            Name
0                               My Love Mine All Mine          Mitski
1                                         Perro Negro       Bad Bunny
2                                              Monaco       Bad Bunny
3                                            Que Onda        Calle 24
4   What Was I Made For? [From The Motion Picture ...   Billie Eilish
5      One Of The Girls (With Jennie, Lily Rose Depp)      The Weeknd
6   Is It Over Now? (Taylor'S Version) (From The V...    Taylor Swift
7                                            Fukumean           Gunna
8                                             Flowers     Miley Cyrus
9                                  Idgaf (Feat. Yeat)           Drake
10                  You’Re Losing Me (From The Vault)    Taylor Swift
11                                           I Know ?    Travis Scott
12  Popular (With Playboi Carti & Mado

In [92]:
# were people happy or sad throughout the decades?

In [93]:
q5 = pd.read_sql("""
    SELECT (CAST(release_year AS INTEGER) / 10) * 10 AS Decade,
           ROUND(AVG(valence), 3) AS AVG_Mood,
           ROUND(AVG(energy), 3) AS AVG_Energy,
           ROUND(AVG(danceability), 3) AS AVG_Danceability,
           COUNT(*) AS Track_Count
    FROM spotify_clean
    WHERE release_year IS NOT NULL
    GROUP BY Decade
    ORDER BY Decade ASC
""", conn)
print("Q5: Average mood, energy and danceability by decade")
print('\n', q5)
print('\n', "Note: 0 = negative/sad, 1 = positive/happy")

Q5: Average mood, energy and danceability by decade

     Decade  AVG_Mood  AVG_Energy  AVG_Danceability  Track_Count
0     1880     0.094       0.073             0.154            4
1     1890     0.584       0.507             0.551           20
2     1900     0.434       0.288             0.679           64
3     1910     0.552       0.454             0.618           13
4     1920     0.303       0.194             0.478            7
5     1930     0.393       0.219             0.674           51
6     1940     0.391       0.256             0.601           80
7     1950     0.480       0.292             0.478         1324
8     1960     0.535       0.408             0.482         3967
9     1970     0.533       0.473             0.489         5819
10    1980     0.520       0.505             0.515         7189
11    1990     0.474       0.477             0.517        18321
12    2000     0.502       0.538             0.558        48875
13    2010     0.457       0.537             0.581

In [94]:
# were people happy or sad in 2023?

In [95]:
q6 = pd.read_sql("""
    SELECT ROUND(AVG(valence), 3) AS AVG_Mood,
        ROUND(AVG(energy), 3) AS AVG_Energy,
        ROUND(AVG(danceability), 3) AS AVG_Danceability
    FROM spotify_clean
    WHERE release_year = 2023
""", conn)

print("Q6: Average mood of songs released in 2023")
print('\n', q6)
print('\n', "Note: 0 = negative/sad, 1 = positive/happy")

Q6: Average mood of songs released in 2023

    AVG_Mood  AVG_Energy  AVG_Danceability
0     0.412       0.527              0.58

 Note: 0 = negative/sad, 1 = positive/happy


In [96]:
q7 = pd.read_sql("""
    SELECT (CAST(release_year AS INTEGER) / 10) * 10 AS Decade,
        explicit as Explicity,
        COUNT(*) as Track_Count
    FROM spotify_clean
    WHERE release_year IS NOT NULL
    AND Explicity IS NOT NULL
    GROUP BY Decade, Explicity
    ORDER BY Decade ASC
""", conn)

print('\n',"Q7: The explicitness of songs throughout the decades")
print('\n',q7)
print('\n', "Note: 0 = False, 1 = True")


 Q7: The explicitness of songs throughout the decades

     Decade  Explicity  Track_Count
0     1880          0            4
1     1890          0           20
2     1900          0           64
3     1910          0           13
4     1920          0            7
5     1930          0           51
6     1940          0           80
7     1950          0         1324
8     1960          0         3967
9     1970          0         5810
10    1970          1            9
11    1980          0         7091
12    1980          1           98
13    1990          0        17151
14    1990          1         1170
15    2000          0        45483
16    2000          1         3382
17    2010          0       124741
18    2010          1        18330
19    2020          0       110155
20    2020          1        35382

 Note: 0 = False, 1 = True


In [97]:
# explicity of songs released in 2023

In [98]:
q8 = pd.read_sql("""
    SELECT explicit as Explicity,
        COUNT(*) as Track_Count
    FROM spotify_clean
    WHERE release_year = 2023
    GROUP BY explicit
""", conn)

print('\n',"Q8: Were songs more explicit in 2023?")
print('\n',q8)
print('\n', "Note: 0 = False, 1 = True")



 Q8: Were songs more explicit in 2023?

    Explicity  Track_Count
0          0        44122
1          1        14623

 Note: 0 = False, 1 = True


In [99]:
# popular old songs in 2023

In [100]:
q9 = pd.read_sql("""
    SELECT track_name, name, release_year, track_popularity
    FROM spotify_clean
    WHERE release_year < 2000
    AND track_popularity > 70
    ORDER BY track_popularity DESC
    LIMIT 20
""", conn)
print('\n',"Q9: What old songs were popular in 2023?")
print('\n',q9)


 Q9: What old songs were popular in 2023?

                                            track_name                   name  \
0                     All I Want For Christmas Is You           Mariah Carey   
1                                    Jingle Bell Rock            Bobby Helms   
2   Wonderful Christmastime - Edited Version / Rem...         Paul McCartney   
3         The Christmas Song (Merry Christmas To You)          Nat King Cole   
4                                               Creep              Radiohead   
5   Let It Snow! Let It Snow! Let It Snow! (With T...          Frank Sinatra   
6         Do They Know It'S Christmas? - 1984 Version               Band Aid   
7                                  Losing My Religion                 R.E.M.   
8                                            Everlong           Foo Fighters   
9                                      Deck The Halls          Nat King Cole   
10                                   Under The Bridge  Red Hot Chili Pepper

In [135]:
q10 = pd.read_sql("""
    SELECT artist as Artist,
           ROUND(AVG(danceability), 3) as AVG_Danceability,
           ROUND(AVG(valence), 3) as AVG_Mood,
           ROUND(AVG(energy), 3) as AVG_Energy
    FROM (
        SELECT name as artist, danceability, valence, energy FROM spotify_clean
        UNION ALL
        SELECT artist_1 as artist, danceability, valence, energy FROM spotify_clean WHERE artist_1 IS NOT NULL
        UNION ALL
        SELECT artist_2 as artist, danceability, valence, energy FROM spotify_clean WHERE artist_2 IS NOT NULL
        UNION ALL
        SELECT artist_3 as artist, danceability, valence, energy FROM spotify_clean WHERE artist_3 IS NOT NULL
        UNION ALL
        SELECT artist_4 as artist, danceability, valence, energy FROM spotify_clean WHERE artist_4 IS NOT NULL
    ) as all_artists
    WHERE artist IN (
        'Taylor Swift', 'Drake', 'Bad Bunny', 'The Weeknd', 'Travis Scott',
        'Kanye West', 'Peso Pluma', 'Eminem', 'Lana Del Rey', 'Jung Kook',
        'Ariana Grande', 'Justin Bieber', 'Rihanna', 'KAROL G', 'SZA',
        '21 Savage', 'Future', 'Junior H', 'Ed Sheeran', 'Billie Eilish'
    )
    GROUP BY artist
    ORDER BY AVG_Mood DESC
""", conn)

print(q10)

           Artist  AVG_Danceability  AVG_Mood  AVG_Energy
0      Peso Pluma             0.801     0.751       0.688
1       Jung Kook             0.758     0.678       0.680
2        Junior H             0.666     0.593       0.780
3          Eminem             0.709     0.535       0.728
4         KAROL G             0.762     0.529       0.743
5   Justin Bieber             0.630     0.495       0.659
6      Ed Sheeran             0.697     0.491       0.614
7       Bad Bunny             0.753     0.484       0.666
8         Rihanna             0.634     0.464       0.713
9   Ariana Grande             0.635     0.453       0.635
10     Kanye West             0.610     0.439       0.626
11   Taylor Swift             0.601     0.416       0.554
12         Future             0.732     0.394       0.587
13   Travis Scott             0.714     0.377       0.647
14      21 Savage             0.766     0.371       0.594
15            SZA             0.656     0.366       0.620
16          Dr

In [119]:
q11 = pd.read_sql("""
    SELECT 
        CASE 
            WHEN release_year = 2023 THEN '2023'
            ELSE CAST((CAST(release_year AS INTEGER) / 10) * 10 AS TEXT) || 's'
        END as Period,
        ROUND(AVG(valence), 3) as AVG_Mood,
        ROUND(AVG(energy), 3) as AVG_Energy,
        ROUND(AVG(danceability), 3) as AVG_Danceability
    FROM spotify_clean
    WHERE release_year IS NOT NULL
    GROUP BY Period
    ORDER BY Period ASC
""", conn)
print(q11)

   Period  AVG_Mood  AVG_Energy  AVG_Danceability
0   1880s     0.094       0.073             0.154
1   1890s     0.584       0.507             0.551
2   1900s     0.434       0.288             0.679
3   1910s     0.552       0.454             0.618
4   1920s     0.303       0.194             0.478
5   1930s     0.393       0.219             0.674
6   1940s     0.391       0.256             0.601
7   1950s     0.480       0.292             0.478
8   1960s     0.535       0.408             0.482
9   1970s     0.533       0.473             0.489
10  1980s     0.520       0.505             0.515
11  1990s     0.474       0.477             0.517
12  2000s     0.502       0.538             0.558
13  2010s     0.457       0.537             0.581
14  2020s     0.432       0.521             0.589
15   2023     0.412       0.527             0.580


In [120]:
q12 = pd.read_sql("""
    SELECT 
        CASE 
            WHEN release_year = 2023 THEN '2023'
            ELSE CAST((CAST(release_year AS INTEGER) / 10) * 10 AS TEXT) || 's'
        END as Period,
        explicit as Explicity,
        COUNT(*) as Track_Count
    FROM spotify_clean
    WHERE release_year IS NOT NULL
    AND explicit IS NOT NULL
    GROUP BY Period, Explicity
    ORDER BY Period ASC
""", conn)

print(q12)

   Period  Explicity  Track_Count
0   1880s          0            4
1   1890s          0           20
2   1900s          0           64
3   1910s          0           13
4   1920s          0            7
5   1930s          0           51
6   1940s          0           80
7   1950s          0         1324
8   1960s          0         3967
9   1970s          0         5810
10  1970s          1            9
11  1980s          0         7091
12  1980s          1           98
13  1990s          0        17151
14  1990s          1         1170
15  2000s          0        45483
16  2000s          1         3382
17  2010s          0       124741
18  2010s          1        18330
19  2020s          0        66033
20  2020s          1        20759
21   2023          0        44122
22   2023          1        14623


In [160]:
# Get top 20 from spotify_artists
q2 = pd.read_sql("""
    SELECT name as Name,
           followers as Followers,
           artist_popularity AS Popularity_Score
    FROM spotify_artists
    GROUP BY followers
    ORDER BY Popularity_Score DESC
    LIMIT 20
""", conn)

# Get audio features including collabs
q10 = pd.read_sql("""
    SELECT artist as Name,
           ROUND(AVG(danceability), 3) as AVG_Danceability,
           ROUND(AVG(valence), 3) as AVG_Mood,
           ROUND(AVG(energy), 3) as AVG_Energy,
           SUM(CASE WHEN explicit = 1 THEN 1 ELSE 0 END) as Explicit_Count,
           SUM(CASE WHEN explicit = 0 THEN 1 ELSE 0 END) as Clean_Count
    FROM (
        SELECT name as artist, danceability, valence, energy, explicit FROM spotify_clean
        UNION ALL
        SELECT artist_1, danceability, valence, energy, explicit FROM spotify_clean WHERE artist_1 IS NOT NULL
        UNION ALL
        SELECT artist_2, danceability, valence, energy, explicit FROM spotify_clean WHERE artist_2 IS NOT NULL
        UNION ALL
        SELECT artist_3, danceability, valence, energy, explicit FROM spotify_clean WHERE artist_3 IS NOT NULL
        UNION ALL
        SELECT artist_4, danceability, valence, energy, explicit FROM spotify_clean WHERE artist_4 IS NOT NULL
    ) as all_artists
    WHERE artist IN (
        'Taylor Swift', 'Drake', 'Bad Bunny', 'The Weeknd', 'Travis Scott',
        'Kanye West', 'Peso Pluma', 'Eminem', 'Lana Del Rey', 'Jung Kook',
        'Ariana Grande', 'Justin Bieber', 'Rihanna', 'KAROL G', 'SZA',
        '21 Savage', 'Future', 'Junior H', 'Ed Sheeran', 'Billie Eilish'
    )
    GROUP BY artist
""", conn)

# Merge in Python
q_artists = q2.merge(q10, on='Name', how='left')

print(q_artists)

             Name  Followers  Popularity_Score  AVG_Danceability  AVG_Mood  \
0    Taylor Swift   95859165               100             0.601     0.416   
1           Drake   83298497                95             0.654     0.355   
2       Bad Bunny   77931484                95             0.753     0.484   
3      The Weeknd   75945958                93             0.576     0.346   
4    Travis Scott   25672786                90             0.714     0.377   
5      Kanye West   22582692                90             0.610     0.439   
6      Peso Pluma    9775920                90             0.801     0.751   
7          Eminem   79891173                89             0.709     0.535   
8    Lana Del Rey   30603103                89             0.467     0.273   
9       Jung Kook   11781452                89             0.758     0.678   
10  Ariana Grande   95710972                88             0.635     0.453   
11  Justin Bieber   75112165                88             0.630

In [ ]:
# exporting queries so i can use them in power bi

In [161]:
q1.to_csv('../exports/q1.csv', index=False)
q2.to_csv('../exports/q2.csv', index=False)
q3.to_csv('../exports/q3.csv', index=False)
q4.to_csv('../exports/q4.csv', index=False)
q5.to_csv('../exports/q5.csv', index=False)
q6.to_csv('../exports/q6.csv', index=False)
q7.to_csv('../exports/q7.csv', index=False)
q8.to_csv('../exports/q8.csv', index=False)
q9.to_csv('../exports/q9.csv', index=False)
q10.to_csv('../exports/q10.csv', index=False)
q11.to_csv('../exports/q11.csv', index=False)
q12.to_csv('../exports/q12.csv', index=False)
q_artists.to_csv('../exports/q_artists.csv', index=False)

print("Queries Successfully Exported")

conn.close()

Queries Successfully Exported
